## Program 8-4

In [1]:
import music21

# "작은 별" 악보를 ABC 표기로 표현
little_star="tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2"
# music21.converter.parse(little_star).show('mid') # 스피커로 연주를 들려줌

In [2]:
import numpy as np

# 계이름과 숫자를 상호 변환하는 표(딕셔너리 자료구조를 사용함)
note2num={'c':1,'d':2,'e':3,'f':4,'g':5,'a':6,'b':7}
num2note={1:'c',2:'d',3:'e',4:'f',5:'g',6:'a',7:'b'}

# ABC 표기를 시계열 데이터로 변환
def abc2timeseries(s):
    notes=s.split(' ')[2:]
    seq=[]
    for i in notes:
        seq.append([note2num[i[0]],int(i[1])])
    return seq

# 시계열 데이터를 ABC 표기로 변환
def timeseries2abc(t):
    s='tinynotation: 4/4'
    for i in t:
        s=s+' '+num2note[i[0]]+str(i[1])
    return s

# 원핫 코드로 변환하는 표
onehot=[[1,2],[2,2],[3,2],[4,2],[5,2],[6,2],[7,2],[1,4],[2,4],[3,4],[4,4],[5,4],[6,4],[7,4],[1,8],[2,8],[3,8],[4,8],[5,8],[6,8],[7,8]]

# 레이블을 원핫 코드로 변환
def to_onehot(l):
    t=[]
    for i in range(len(l)):
        a=np.zeros(len(onehot))
        a[onehot.index(list(l[i]))]=1.0
        t.append(a)
    return np.array(t)

# 시계열 데이터를 훈련 집합으로 자름
def seq2dataset(seq,window,horizon):
    X=[]; Y=[]
    for i in range(len(seq)-(window+horizon)+1):
        x=seq[i:(i+window)]
        y=(seq[i+window+horizon-1])
        X.append(x); Y.append(y)
    return np.array(X), np.array(Y)

w=8 # 윈도우 크기
h=1 # 수평선 계수

seq=abc2timeseries(little_star)
X,Y=seq2dataset(seq,w,h)
print(X.shape,Y.shape)
print(X[0],Y[0])

(34, 8, 2) (34, 2)
[[1 4]
 [1 4]
 [5 4]
 [5 4]
 [6 4]
 [6 4]
 [5 2]
 [4 4]] [4 4]


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import tensorflow as tf

In [4]:
# 훈련 집합 구축
split=int(len(X)*1.0) # 100%를 훈련집합으로 사용
x_train=X[0:split]; y_train=Y[0:split]
y_train=to_onehot(y_train)

# LSTM 모델 설계와 학습
model=Sequential()
model.add(LSTM(units=128,activation='relu',input_shape=x_train[0].shape))
model.add(Dense(y_train.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=200,batch_size=1,verbose=2)

Epoch 1/200
34/34 - 1s - loss: 2.9393 - accuracy: 0.0588 - 765ms/epoch - 23ms/step
Epoch 2/200
34/34 - 0s - loss: 2.2818 - accuracy: 0.2059 - 195ms/epoch - 6ms/step
Epoch 3/200
34/34 - 0s - loss: 2.1162 - accuracy: 0.1765 - 194ms/epoch - 6ms/step
Epoch 4/200
34/34 - 0s - loss: 1.9979 - accuracy: 0.2647 - 189ms/epoch - 6ms/step
Epoch 5/200
34/34 - 0s - loss: 1.8892 - accuracy: 0.2647 - 193ms/epoch - 6ms/step
Epoch 6/200
34/34 - 0s - loss: 1.8101 - accuracy: 0.3529 - 191ms/epoch - 6ms/step
Epoch 7/200
34/34 - 0s - loss: 1.7066 - accuracy: 0.3235 - 192ms/epoch - 6ms/step
Epoch 8/200
34/34 - 0s - loss: 1.7291 - accuracy: 0.3529 - 193ms/epoch - 6ms/step
Epoch 9/200
34/34 - 0s - loss: 1.5436 - accuracy: 0.3824 - 195ms/epoch - 6ms/step
Epoch 10/200
34/34 - 0s - loss: 1.5488 - accuracy: 0.3529 - 194ms/epoch - 6ms/step
Epoch 11/200
34/34 - 0s - loss: 1.4301 - accuracy: 0.4706 - 194ms/epoch - 6ms/step
Epoch 12/200
34/34 - 0s - loss: 1.3828 - accuracy: 0.5000 - 191ms/epoch - 6ms/step
Epoch 13/200

In [5]:
x_train[0],y_train[0]

(array([[1, 4],
        [1, 4],
        [5, 4],
        [5, 4],
        [6, 4],
        [6, 4],
        [5, 2],
        [4, 4]]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]))

In [6]:
# 학습된 모델로 편곡을 하는 함수(first_measure: 첫 소절, duration: 생성될 곡의 길이)
def arranging_music(model,first_measure,duration):
    music=first_measure
    for i in range(duration):
        p=model.predict(np.float32(np.expand_dims(music[-w:],axis=0)))
        music=np.append(music,[onehot[np.argmax(p)]],axis=0)
    return timeseries2abc(music)

In [7]:
new_song=arranging_music(model,x_train[0],50)

print(new_song)
# music21.converter.parse(new_song).show('mid')

1/1 [==============================] - 0s 11ms/step
tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4


In [8]:
new_song=arranging_music(model,x_train[0],100)

print(new_song)
# music21.converter.parse(new_song).show('mid')

1/1 [==============================] - 0s 13ms/step
tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4


In [9]:
new_song=arranging_music(model,[[5,4],[3,4],[3,2],[4,4],[2,4],[2,2],[1,4],[2,4]],50)

print(new_song)

1/1 [==============================] - 0s 12ms/step
tinynotation: 4/4 g4 e4 e2 f4 d4 d2 c4 d4 g4 g4 a4 a4 g2 f4 e4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4


## Program 8-5

In [10]:
# "작은 별", "봄나들이", "나비야 나비야" 악보를 ABC 표기로 표현
little_star="tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2"
spring_picnic="tinynotation: 4/8 g8 e8 g8 e8 g8 a8 g4 e8 g8 e8 c8 d8 e8 c4 g8 e8 g8 e8 g8 a8 g4 b8 a8 g8 e8 d8 e8 c4"
butterfly="tinynotation: 2/4 g8 e8 e4 f8 d8 d4 c8 d8 e8 f8 g8 g8 g4 g8 e8 e8 e8 f8 d8 d4 c8 e8 g8 g8 e8 e8 e4 d8 d8 d8 d8 d8 e8 f4 e8 e8 e8 e8 e8 f8 g4 g8 e8 e4 f8 d8 d4 c8 e8 g8 g8 e8 e8 e4"

In [11]:
# 세 곡을 시계열로 변환하고 결합
seq1=abc2timeseries(little_star)
seq2=abc2timeseries(butterfly)
seq3=abc2timeseries(spring_picnic)
seq=seq1+seq2+seq3

X,Y = seq2dataset(seq,w,h)

In [12]:
# 훈련 집합 구축
split=int(len(X)*1.0) # 100%를 훈련집합으로 사용
x_train=X[0:split]; y_train=Y[0:split]
y_train=to_onehot(y_train)

# LSTM 모델 설계와 학습
model2=Sequential()
model2.add(LSTM(units=128,activation='relu',input_shape=x_train[0].shape))
model2.add(Dense(y_train.shape[1],activation='softmax'))
model2.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model2.fit(x_train,y_train,epochs=200,batch_size=1,verbose=2)

Epoch 1/200
116/116 - 1s - loss: 2.6729 - accuracy: 0.1810 - 1s/epoch - 10ms/step
Epoch 2/200
116/116 - 1s - loss: 2.3130 - accuracy: 0.2241 - 658ms/epoch - 6ms/step
Epoch 3/200
116/116 - 1s - loss: 2.2250 - accuracy: 0.2414 - 665ms/epoch - 6ms/step
Epoch 4/200
116/116 - 1s - loss: 2.1989 - accuracy: 0.2845 - 654ms/epoch - 6ms/step
Epoch 5/200
116/116 - 1s - loss: 2.1459 - accuracy: 0.2500 - 667ms/epoch - 6ms/step
Epoch 6/200
116/116 - 1s - loss: 2.1045 - accuracy: 0.2931 - 662ms/epoch - 6ms/step
Epoch 7/200
116/116 - 1s - loss: 2.0128 - accuracy: 0.3276 - 674ms/epoch - 6ms/step
Epoch 8/200
116/116 - 1s - loss: 1.9747 - accuracy: 0.3276 - 673ms/epoch - 6ms/step
Epoch 9/200
116/116 - 1s - loss: 1.9605 - accuracy: 0.3103 - 662ms/epoch - 6ms/step
Epoch 10/200
116/116 - 1s - loss: 1.8832 - accuracy: 0.3190 - 658ms/epoch - 6ms/step
Epoch 11/200
116/116 - 1s - loss: 1.8869 - accuracy: 0.3448 - 671ms/epoch - 6ms/step
Epoch 12/200
116/116 - 1s - loss: 1.8196 - accuracy: 0.3621 - 669ms/epoch - 

In [13]:
new_song=arranging_music(model2,x_train[0],50)

print(new_song)

1/1 [==============================] - 0s 13ms/step
tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g8 e8 e4 f8 d8 d4 c8 d8 e8 f8 g8 g8 g4 g8 e8 e8 e8 f8 d8 d4 c8 e8 g8 g8 e8 e8 e4 d8 d8 d8 d8 d8 e8 f4 e8 e8 e8 e8 e8 f8 g4 g8 e8 e4
